# Burgers Fortran Implementation

This is a version used to run on the LNCC SDumont.

Based on:

- https://github.com/maziarraissi/PINNs/tree/master/appendix/continuous_time_inference%20(Burgers)
- https://people.sc.fsu.edu/~jburkardt/f_src/burgers_solution/burgers_solution.html

Evaluates exact solutions of the time-dependent 1D viscous Burgers equation. The form of the Burgers equation considered here is

$ \large \frac{\partial u}{\partial t} + u \frac{\partial u}{\partial x} = \nu \frac{\partial^2u}{\partial x^2} \normalsize,
\quad x \in [-1,1], \ t \in [0, 1] \\
\text{IC: } u(0, x) = - sen(\pi x) \\
\text{BC: } u(t, -1) = u(t, 1) = 0 \\
\text{Viscosity: }  \large \nu = \frac{0.01}{\pi}
$

- approximation using a Gauss-Hermite quadrature rule
- evaluate the exact solution at a user-specified set of points, using the quadrature rule
- internally, the order of this quadrature rule is set to 8

Input

- nu : the viscosity
- vtn : the number of time grid points
- vxn : the number of spatial grid points
- vx(vxn) : the spatial grid points
- vt(vtn) : the time grid points

Output

- vu(vxn,vtn) : the solution of the Burgers equation at each space and time grid point

In [64]:
%%writefile burgers.f90
program main
use omp_lib
implicit none
    integer, parameter :: vtn = 100  ! outer loop
    integer, parameter :: vxn = 256  ! inner loop
    integer            :: i0, i1
    double precision   :: nu, thi, tlo, vu(vxn, vtn), vt(vtn), vx(vxn), xhi,  &
                          xlo , t0, t1
    ! r8 means real type number
    double precision, parameter :: r8_pi = 3.141592653589793D+00
    character(len=80) filename

    t0 = omp_get_wtime()    ! time measurement
    i1 = 0
    do i0 = 0, 999
        i1 = i1 + 1
        nu = 0.01D+00/r8_pi
        xlo = -1.0D+00
        xhi = +1.0D+00
        call r8vec_even(vxn, xlo, xhi, vx)
        tlo = 0
        thi = 0.99D+00
        call r8vec_even(vtn, tlo, thi, vt)
        call burgers_viscous_time_exact1(nu, vxn, vx, vtn, vt, vu)
    enddo
    t1 = omp_get_wtime()    ! time measurement
    print "('Elapsed time: 'f0.4)", t1 - t0
    filename = 'burgers03.txt'
    call r8mat_write(filename, vxn, vtn, vu)
end


!
! evaluates solution to the Burgers equation.
!
subroutine burgers_viscous_time_exact1(nu, vxn, vx, vtn, vt, vu)
implicit none
    integer :: vxn, vtn, qi, vti, vxi
    integer, parameter :: qn = 8
    double precision :: nu, vx(vxn), vt(vtn), vu(vxn, vtn), bot, c, qw(qn),  &
                        qx(qn), top
    double precision, parameter :: r8_pi = 3.141592653589793D+00

    !  Compute the rule.
    call hermite_ek_compute(qn, qx, qw)
    
    !  Evaluate U(X,T) for later times.
    !----------------------------------------
    do vti = 1, vtn
        if (vt(vti) < 1.0D-5) then
            do vxi = 1, vxn
                vu(vxi, vti) = -sin(r8_pi*vx(vxi))
            end do
        else
            !----------------------------------------
            !$OMP PARALLEL DO PRIVATE(vxi, top, bot, qi, c)  &
            !$OMP             SHARED (vxn, vu)
            do vxi = 1, vxn
                top = 0
                bot = 0
                do qi = 1, qn
                    c = 2.0D+00*sqrt(nu*vt(vti))
                    top = top - qw(qi)*c*sin(r8_pi*(vx(vxi) - c*qx(qi))) &
                        *exp(-cos(r8_pi*(vx(vxi) - c*qx(qi))) &
                        /(2.0D+00*r8_pi*nu))
                    bot = bot + qw(qi)*c &
                        *exp(-cos(r8_pi*(vx(vxi) - c*qx(qi))) &
                        /(2.0D+00*r8_pi*nu))
                    vu(vxi, vti) = top/bot
                end do
            end do
            !$OMP END PARALLEL DO
        end if
    end do
end subroutine


!
! Computes a Gauss-Hermite quadrature rule
!
subroutine hermite_ek_compute(n, x, w)
implicit none
    integer :: n, i
    double precision :: x(n), w(n), bj(n), zemu
    ! Define the zero-th moment.
    zemu = gamma(1.0D+00/2.0D+00)
    ! Define the Jacobi matrix.
    do i = 1, n
        bj(i) = real(i, kind=8)/2.0D+00
    end do
    bj(1:n) = sqrt(bj(1:n))
    x(1:n) = 0
    w(1) = sqrt(zemu)
    w(2:n) = 0
    ! Diagonalize the Jacobi matrix.
    call imtqlx(n, x, bj, w)
    w(1:n) = w(1:n)**2
end subroutine


!
! Diagonalizes a symmetric tridiagonal matrix
!
subroutine imtqlx(n, d, e, z)
implicit none
    integer :: n, i, ii, j, k, l, m, mml
    integer, parameter :: itn = 30
    double precision :: b, c, d(n), e(n), f, g, p, prec, r, s, z(n)

    prec = epsilon(prec)
    if (n == 1) then
        return
    end if
    e(n) = 0
    do l = 1, n
        j = 0
        do
            do m = l, n
                if (m == n) then
                    exit
                end if
                if (abs(e(m)) <= prec*(abs(d(m)) + abs(d(m + 1)))) then
                    exit
                end if
            end do
            p = d(l)
            if (m == l) then
                exit
            end if
            if (itn <= j) then
                write (*, '(a)') ' '
                write (*, '(a)') 'IMTQLX - Fatal error!'
                write (*, '(a)') '  Iteration limit exceeded.'
                write (*, '(a,i8)') '  J = ', j
                write (*, '(a,i8)') '  L = ', l
                write (*, '(a,i8)') '  M = ', m
                write (*, '(a,i8)') '  N = ', n
                stop
            end if
            j = j + 1
            g = (d(l + 1) - p)/(2.0D+00*e(l))
            r = sqrt(g*g + 1.0D+00)
            g = d(m) - p + e(l)/(g + sign(r, g))
            s = 1.0D+00
            c = 1.0D+00
            p = 0
            mml = m - l
            do ii = 1, mml
                i = m - ii
                f = s*e(i)
                b = c*e(i)
                if (abs(g) <= abs(f)) then
                    c = g/f
                    r = sqrt(c*c + 1.0D+00)
                    e(i + 1) = f*r
                    s = 1.0D+00/r
                    c = c*s
                else
                    s = f/g
                    r = sqrt(s*s + 1.0D+00)
                    e(i + 1) = g*r
                    c = 1.0D+00/r
                    s = s*c
                end if
                g = d(i + 1) - p
                r = (d(i) - g)*s + 2.0D+00*c*b
                p = s*r
                d(i + 1) = g + p
                g = c*r - b
                f = z(i + 1)
                z(i + 1) = s*z(i) + c*f
                z(i) = c*z(i) - s*f
            end do
            d(l) = d(l) - p
            e(l) = g
            e(m) = 0
        end do
    end do
    !  Sorting.
    do ii = 2, n
        i = ii - 1
        k = i
        p = d(i)
        do j = ii, n
            if (d(j) < p) then
                k = j
                p = d(j)
            end if
        end do
        if (k /= i) then
            d(k) = d(i)
            d(i) = p
            p = z(i)
            z(i) = z(k)
            z(k) = p
        end if
    end do
end subroutine


!
! Returns an vector of R8 of evenly spaced values
!
subroutine r8vec_even(n, alo, ahi, a)
implicit none
    integer :: n, i
    double precision :: a(n), ahi, alo

    if (n == 1) then
        a(1) = 0.5D+00 * (alo + ahi)
    else
        do i = 1, n
            a(i) = ( dble(n - i) * alo   &
                   + dble(i - 1) * ahi)  &
                   / dble(n - 1)
        end do
    end if
end subroutine


!
! Writes an array of R8 values to a file
!
subroutine r8mat_write(output_filename, m, n, table)
implicit none
    integer :: m, n, j, output_status, output_unit
    character(len=*) :: output_filename
    character(len=30) :: string
    double precision :: table(m, n)
    
    !
    !  Open the file.
    !
    call get_unit(output_unit)
    open (unit=output_unit, file=output_filename, &
        status='replace', iostat=output_status)
    if (output_status /= 0) then
        write (*, '(a)') ' '
        write (*, '(a)') 'R8MAT_WRITE - Fatal error!'
        write (*, '(a,i8)') '  Could not open the output file "'// &
            trim(output_filename)//'" on unit ', output_unit
        output_unit = -1
        stop
    end if
    !  Create a format string.
    !  For less precision in the output file, try:
    !     '(', m, 'g', 14, '.', 6, ')'
    if (0 < m .and. 0 < n) then
        write (string, '(a1,i8,a1,i8,a1,i8,a1)') '(', m, 'g', 24, '.', 16, ')'
        !  Write the data.
        do j = 1, n
            write (output_unit, string) table(1:m, j)
        end do
    end if
    !  Close the file.
    close (unit=output_unit)
end subroutine


!
! returns a free FORTRAN unit number which is not currently
! associated with an I/O device
!
subroutine get_unit(iunit)
implicit none
    integer i, ios, iunit
    logical lopen
    iunit = 0
    do i = 1, 99
        if (i /= 5 .and. i /= 6 .and. i /= 9) then
            inquire (unit=i, opened=lopen, iostat=ios)
            if (ios == 0) then
                if (.not. lopen) then
                    iunit = i
                    return
                end if
            end if
        end if
    end do
end subroutine

Overwriting burgers.f90


Checking:

In [65]:
! gfortran -O3 -fopenmp burgers.f90 -o burgers

In [68]:
! OMP_NUM_THREADS=1 ./burgers

Elapsed time: 21.2476


In [69]:
! OMP_NUM_THREADS=4 ./burgers

Elapsed time: 6.6035


In [70]:
! OMP_NUM_THREADS=8 ./burgers

Elapsed time: 3.6634


Copy to /scrath in order to run on node:

In [71]:
%%bash
BASE=/scratch${HOME#/prj}/421
cp burgers $BASE

Slurm script:

In [72]:
%%writefile burgf90.srm
#!/bin/bash
#SBATCH --job-name burgf90      # SLURM_JOB_NAME
#SBATCH --partition cpu_dev     # SLURM_JOB_PARTITION
#SBATCH --nodes=1               # SLURM_JOB_NUM_NODES
#SBATCH --ntasks-per-node=1     # SLURM_NTASKS_PER_NODE
#SBATCH --cpus-per-task=1       # SLURM_CPUS_PER_TASK  $OMP_THREADS
#SBATCH --time=00:05:00         # Limit execution time

# VARIABLES OF INTEREST IN THE SLURM ENVIRONMENT
# <https://slurm.schedmd.com/sbatch.html>
# SLURM_PROCID
#     The MPI rank (or relative process ID) of the current process.
# SLURM_LOCALID
#     Node local task ID for the process within a job.
# SLURM_NODEID
#     ID of the nodes allocated. 

echo '========================================'
echo '- Job ID:' $SLURM_JOB_ID
echo '- # of nodes in the job:' $SLURM_JOB_NUM_NODES
echo '- # of tasks per node:' $SLURM_NTASKS_PER_NODE
echo '- # of tasks:' $SLURM_NTASKS
echo '- # of cpus per task:' $SLURM_CPUS_PER_TASK
echo '- Dir from which sbatch was invoked:' ${SLURM_SUBMIT_DIR##*/}
echo -n '- Nodes allocated to the job: '
nodeset -e $SLURM_JOB_NODELIST

# load the Python environment
SCR=/scratch${PWD#/prj}
BASE=/scratch${HOME#/prj}/miniconda3
source $BASE/etc/profile.d/conda.sh
conda activate
conda activate tf1
cd $SCR

# run
echo -n '<1. starting> ' && date
echo "OMP_NUM_THREADS=$SLURM_CPUS_PER_TASK"
echo '-- output -----------------------------'
               
export OMP_NUM_THREADS=$SLURM_CPUS_PER_TASK
srun -n $SLURM_NTASKS -c $SLURM_CPUS_PER_TASK time ./burgers
               
echo '-- end --------------------------------'
echo -n '<2. quit> ' && date

Overwriting burgf90.srm


Code to submit a job to the execution queue, and wait for the result:

In [3]:
import time

def runtest(tasks):
    sub = !sbatch --cpus-per-task={tasks} burgf90.srm
    print(sub[0], end='.')
    job = sub[0].replace('Submitted batch job ','')
    c = [job]
    while job in c:
        time.sleep(10)
        print(end='.')
        c = !squeue --job {job} --noheader --format "%i"
    print('')
    out = !echo /scratch${PWD#/prj}/slurm-
    %cat {out[0] + job}.out

## OMP_NUM_THREADS=1

In [74]:
runtest(1)

Submitted batch job 10723667.....
- Job ID: 10723667
- # of nodes in the job: 1
- # of tasks per node: 1
- # of tasks: 1
- # of cpus per task: 1
- Dir from which sbatch was invoked: 421
- Nodes allocated to the job: sdumont1000
<1. starting> Sáb Dez 10 20:40:26 -03 2022
OMP_NUM_THREADS=1
-- output -----------------------------
Elapsed time: 20.0968
20.08user 0.03system 0:20.15elapsed 99%CPU (0avgtext+0avgdata 1592maxresident)k
3152inputs+1208outputs (3major+713minor)pagefaults 0swaps
-- end --------------------------------
<2. quit>                    Sáb Dez 10 20:40:46 -03 2022


## OMP_NUM_THREADS=4

In [75]:
runtest(4)

Submitted batch job 10723668...
- Job ID: 10723668
- # of nodes in the job: 1
- # of tasks per node: 1
- # of tasks: 1
- # of cpus per task: 4
- Dir from which sbatch was invoked: 421
- Nodes allocated to the job: sdumont1000
<1. starting> Sáb Dez 10 20:41:01 -03 2022
OMP_NUM_THREADS=4
-- output -----------------------------
Elapsed time: 6.2506
24.97user 0.05system 0:06.28elapsed 398%CPU (0avgtext+0avgdata 1636maxresident)k
3152inputs+1208outputs (3major+721minor)pagefaults 0swaps
-- end --------------------------------
<2. quit>                    Sáb Dez 10 20:41:11 -03 2022


## OMP_NUM_THREADS=8

In [76]:
runtest(8)

Submitted batch job 10723669...
- Job ID: 10723669
- # of nodes in the job: 1
- # of tasks per node: 1
- # of tasks: 1
- # of cpus per task: 8
- Dir from which sbatch was invoked: 421
- Nodes allocated to the job: sdumont1000
<1. starting> Sáb Dez 10 20:41:21 -03 2022
OMP_NUM_THREADS=8
-- output -----------------------------
Elapsed time: 3.5162
28.09user 0.07system 0:03.55elapsed 791%CPU (0avgtext+0avgdata 1676maxresident)k
3152inputs+1208outputs (3major+1269minor)pagefaults 0swaps
-- end --------------------------------
<2. quit>                    Sáb Dez 10 20:41:26 -03 2022


## OMP_NUM_THREADS=16

In [3]:
runtest(16)

Submitted batch job 10723677...
- Job ID: 10723677
- # of nodes in the job: 1
- # of tasks per node: 1
- # of tasks: 1
- # of cpus per task: 16
- Dir from which sbatch was invoked: 421
- Nodes allocated to the job: sdumont1000
<1. starting> Sáb Dez 10 21:04:32 -03 2022
OMP_NUM_THREADS=16
-- output -----------------------------
Elapsed time: 2.3667
37.46user 0.09system 0:02.42elapsed 1549%CPU (0avgtext+0avgdata 1744maxresident)k
3152inputs+1208outputs (3major+1188minor)pagefaults 0swaps
-- end --------------------------------
<2. quit>                    Sáb Dez 10 21:04:35 -03 2022


## OMP_NUM_THREADS=24

In [5]:
runtest(24)

Submitted batch job 10723680...
- Job ID: 10723680
- # of nodes in the job: 1
- # of tasks per node: 1
- # of tasks: 1
- # of cpus per task: 24
- Dir from which sbatch was invoked: 421
- Nodes allocated to the job: sdumont1000
<1. starting> Sáb Dez 10 21:06:23 -03 2022
OMP_NUM_THREADS=24
-- output -----------------------------
Elapsed time: 2.0607
49.18user 0.12system 0:02.12elapsed 2319%CPU (0avgtext+0avgdata 1820maxresident)k
3152inputs+1208outputs (3major+2289minor)pagefaults 0swaps
-- end --------------------------------
<2. quit>                    Sáb Dez 10 21:06:25 -03 2022


# Execution 2

(there are 3 measurements of elapsed time in total)

In [4]:
runtest(1)
runtest(4)
runtest(8)
runtest(16)
runtest(24)

Submitted batch job 10724679.....
- Job ID: 10724679
- # of nodes in the job: 1
- # of tasks per node: 1
- # of tasks: 1
- # of cpus per task: 1
- Dir from which sbatch was invoked: 421
- Nodes allocated to the job: sdumont1487
<1. starting> Dom Dez 11 21:59:01 -03 2022
OMP_NUM_THREADS=1
-- output -----------------------------
Elapsed time: 20.0863
20.05user 0.04system 0:20.14elapsed 99%CPU (0avgtext+0avgdata 1592maxresident)k
3696inputs+1208outputs (3major+712minor)pagefaults 0swaps
-- end --------------------------------
<2. quit>                    Dom Dez 11 21:59:22 -03 2022
Submitted batch job 10724683...
- Job ID: 10724683
- # of nodes in the job: 1
- # of tasks per node: 1
- # of tasks: 1
- # of cpus per task: 4
- Dir from which sbatch was invoked: 421
- Nodes allocated to the job: sdumont1487
<1. starting> Dom Dez 11 21:59:35 -03 2022
OMP_NUM_THREADS=4
-- output -----------------------------
Elapsed time: 6.2738
25.05user 0.06system 0:06.30elapsed 398%CPU (0avgtext+0avgdata 16

# Execution 3

In [5]:
runtest(1)
runtest(4)
runtest(8)
runtest(16)
runtest(24)

Submitted batch job 10724687....
- Job ID: 10724687
- # of nodes in the job: 1
- # of tasks per node: 1
- # of tasks: 1
- # of cpus per task: 1
- Dir from which sbatch was invoked: 421
- Nodes allocated to the job: sdumont1487
<1. starting> Dom Dez 11 22:00:29 -03 2022
OMP_NUM_THREADS=1
-- output -----------------------------
Elapsed time: 20.2822
20.27user 0.02system 0:20.31elapsed 99%CPU (0avgtext+0avgdata 1592maxresident)k
3400inputs+1208outputs (3major+713minor)pagefaults 0swaps
-- end --------------------------------
<2. quit>                    Dom Dez 11 22:00:49 -03 2022
Submitted batch job 10724688...
- Job ID: 10724688
- # of nodes in the job: 1
- # of tasks per node: 1
- # of tasks: 1
- # of cpus per task: 4
- Dir from which sbatch was invoked: 421
- Nodes allocated to the job: sdumont1001
<1. starting> Dom Dez 11 22:00:57 -03 2022
OMP_NUM_THREADS=4
-- output -----------------------------
Elapsed time: 6.9311
27.67user 0.07system 0:06.97elapsed 398%CPU (0avgtext+0avgdata 163

---

## Versions

In [37]:
! gfortran --version

GNU Fortran (GCC) 4.8.5 20150623 (Red Hat 4.8.5-44)
Copyright (C) 2015 Free Software Foundation, Inc.

GNU Fortran comes with NO WARRANTY, to the extent permitted by law.
You may redistribute copies of GNU Fortran
under the terms of the GNU General Public License.
For more information about these matters, see the file named COPYING



In [1]:
! ldd burgers

	linux-vdso.so.1 =>  (0x00007ffd875c5000)
	libgfortran.so.3 => /lib64/libgfortran.so.3 (0x00007f649f7ef000)
	libm.so.6 => /lib64/libm.so.6 (0x00007f649f4ed000)
	libgomp.so.1 => /lib64/libgomp.so.1 (0x00007f649f2c7000)
	libgcc_s.so.1 => /lib64/libgcc_s.so.1 (0x00007f649f0b1000)
	libquadmath.so.0 => /lib64/libquadmath.so.0 (0x00007f649ee75000)
	libpthread.so.0 => /lib64/libpthread.so.0 (0x00007f649ec59000)
	libc.so.6 => /lib64/libc.so.6 (0x00007f649e88b000)
	/lib64/ld-linux-x86-64.so.2 (0x00007f649fb11000)


In [2]:
! echo $PATH

/scratch/ampemi/xxxx.yyyy/miniconda3/envs/tf1/bin:/scratch/ampemi/xxxx.yyyy/miniconda3/condabin:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/opt/ibutils/bin:/prj/ampemi/xxxx.yyyy/.local/bin:/prj/ampemi/xxxx.yyyy/bin


In [3]:
! echo _OPENMP | gcc -fopenmp -E -x c - | tail -1

201107


from https://gcc.gnu.org/wiki/openmp : 201107 --> OpenMP v3.1